In [43]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

# Uses BeautifulSoup to scrap the url variable and separate the object 'table'
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
s = requests.get(url).text
soup = BeautifulSoup(s, 'xml')
table = soup.find('table')

# Sets up the base of the dataframe df
df = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])

# Parses the object 'table' into the dataframe 'df'
for tr in table.find_all('tr'):
    row = []
    for td in tr.find_all('td'):
        row.append(td.text.strip())
    if len(row)==3:
        df.loc[len(df)] = row
        
# Drops any rows where the Borough is 'Not assigned'
df = df[df['Borough'] != 'Not assigned']

# Replaces the Neighborhood name with the Borough when Neighborhood is 'Not assigned'
i = 0
while i < len(df):
    if df.iloc[i,2] == 'Not assigned':
        df.iloc[i,2] = df.iloc[i,1]
    i += 1

# Collapses Neighborhoods per PostalCode down to the single record
df = pd.DataFrame(df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: "%s" % ',  '.join(x)))
df.rename(columns={'Neighborhood':'Neighborhoods'}, inplace=True)
df = df.reset_index()

# Reads the geospatial coordinates in the dataframe 'geo' and preps for joining
geo = pd.read_csv('Geospatial_Coordinates.csv')
geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

# Merges the geospatial coordinates from the dataframe 'geo' with the base dataframe 'df'
df = pd.merge(df, geo, how='left', on='PostalCode')
df.head()

,PostalCode,Borough,Neighborhoods,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
